<h1><center> Configuring igpu for xserver and Nvidia gpu for CUDA Ubuntu 18.04 </center></h1>

### This step by step guide will help you install Nvidia GPU to work on CUDA only, and for the integrated gpu to take care of the displays.
Bionic Beaver (Ubuntu 18.04) is shipped with both the traditional Xorg graphics stack as well as the newer Wayland based stack, but Xorg will be the default. This [article](https://blog.ubuntu.com/2018/01/26/bionic-beaver-18-04-lts-to-use-xorg-by-default) written by Will Cooke a Desktop Engineering Manager explains why Xorg is not the default on Bionic Beaver, in contrast with the previous Ubuntu 17.04 version.

In general Xorg is the X Window server which allows users to have a graphical environment at their fingertips. A thorough guide that explains what Xorg is can be found [here](https://wiki.gentoo.org/wiki/Xorg/Guide). [Wayland](https://wayland.freedesktop.org/) is intended as a simpler replacement for X, easier to develop and maintain.


To check whether your Ubuntu is running on Wayland or Xorg, open a terminal use:

        > cat /etc/gdm3/custom.conf
To disable Wayland access the configuration file with an editor of your choice using **sudo** and **un-comment** the line, and reboot the system to apply changes.
    
        > #WaylandEnable=false


To check what hardware is in our system:

    > sudo lshw -c display

![Display](images/displays.png)

From the image above we can see that there are 2 hardware devices related to the display.

What is important for us is to see that the driver installed for the GPU is the [nouveau](https://nouveau.freedesktop.org/wiki/) driver which is a free and open-source graphics device driver for nvidia video cards. You can also check which type of driver you are using from the software and update center under additional drivers.

![Drivers](images/SU_ss.png)



<h1><center> 1 - Installing Via GUI </center></h1>

On Ubuntu 18.04, setting up the nvidia driver is fairly simple. It can be done through the graphical user interface above. You can compare the driver number with the one listed on the [nvidia website](https://www.nvidia.com/Download/index.aspx) by choosing your system details. It is recommended to **disable** secure boot if you are using UEFI firmware, some problems might be encountered if it is enabled.




<h1><center> 2 - Installing Via Command line</center></h1>

First by adding the Nvidia PPA:

    > sudo add-apt-repository ppa:graphics-drivers/ppa
Then:

    > sudo apt install nvidia-driver-**driver-version**
    
In our case:
    
    > sudo apt install nvidia-driver-390

<h1><center> Installing CUDA on Ubuntu 18.04</center></h1>

To check which cuda version is compatible with the driver you just installed please check this [post](https://stackoverflow.com/questions/30820513/what-is-version-of-cuda-for-nvidia-304-125/30820690#30820690). 

In our case for a 390 driver version, CUDA 9.1 is the compatible one.

The CUDA development environment relies on tight integration with the host development environment, including the host compiler and C runtime libraries and is therefore only supported on distribution versions that have been qualified for this CUDA Toolkit release. Full documentation can be found [here](https://docs.nvidia.com/cuda/archive/9.1/cuda-installation-guide-linux/index.html).

![CUDA](images/cuda_dependencies.png)

Since the default gcc version of ubuntu 18.04 is 7.3.0, it needs to be changed. To check what version of gcc you are using:

    > gcc --version
    
![gcc](images/gcc.png)

The default kernel version of ubuntu 18.04 is 4.15. To check the version use:
    
    > uname -r

![kernel](images/kernel.png)

To check the ldd version:
    
    > ldd --version

![ldd](images/ldd.png)

A solution for the gcc/g++ compatibility is to remove the gcc/g++7.3 and install the correct version required, but a yet better solution is to use the update-alternatives that allows you to switch between versions.

Remove current alternatives:


    > sudo update-alternatives --remove-all gcc 
    > sudo update-alternatives --remove-all g++
    
Install packages:

    > sudo apt-get install gcc-6 g++-6
    
Install Alternatives:
       
    > update-alternatives: --install needs (link) (name) (path) (priority)

       
    > sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-6 10
    > sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 20
    
    > sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-6 10
    > sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-7 20
    
    > sudo update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
    > sudo update-alternatives --set cc /usr/bin/gcc
    > sudo update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
    > sudo update-alternatives --set c++ /usr/bin/g++
    
Configure Alternatives:

    > sudo update-alternatives --config gcc
    > sudo update-alternatives --config g++
    
Installing other CUDA dependencies:

    > sudo apt-get install freeglut3 freeglut3-dev libxi-dev libxmu-dev

<h2><center> CUDA run file installer </center></h2>

Since Nvidia did not include an installer for Ubuntu 18.04, we are going to use the run files of CUDA toolkit 9.1 for ubuntu 17.04.

Head to the [CUDA toolkit archive](https://developer.nvidia.com/cuda-91-download-archive) and download the runfile installer and all the patches.

![cuda_runfile](images/cuda_runfile.png)

Make sure that driver version on the CUDA runfile name is the one supported (It is not necessary for them to be identical). For example here it is 387.26 but our driver version is 390.xx. You can then run the runfile you just downloaded using:

    > sudo sh cuda_9.1.85_387.26_linux.run
    
    
You will be prompted with several questions one of which is:


> Install NVIDIA Accelerated Graphics Driver for Linux-x86_64 387.26? y/n/q


    > Make sure to answer **n**, as we already installed the nvidia driver
    
  
> Enter CUDA Samples Location:

        > You can set it to **/usr/local/cuda-9.1** if you don't want it in your home directory

Install the patches by order:

    > sudo sh cuda_9.1.85.1_linux.run
    > sudo sh cuda_9.1.85.2_linux.run
    > sudo sh cuda_9.1.85.3_linux.run
    


<h2><center> Setting up environment variables (System wide) </center></h2>

Create the file cuda.sh:

    > sudo touch /etc/profile.d/cuda.sh

Add the following commands inside the cuda.sh file:

    > echo $'export PATH=$PATH:/usr/local/cuda/bin \nexport CUDADIR=/usr/local/cuda' | sudo tee --append /etc/profile.d/cuda.sh
    
Create the following file as well:

    > sudo touch /etc/ld.so.conf.d/cuda.conf
    
Add the following command:

    > echo '/usr/local/cuda/lib64' | sudo tee --append /etc/ld.so.conf.d/cuda.conf
    
Run:

    > sudo ldconfig
    
And finally:

    > source /etc/profile.d/cuda.sh

Credits:

GCC alternatives:

https://askubuntu.com/questions/26498/how-to-choose-the-default-gcc-and-g-version

CUDA installation: 

https://www.pugetsystems.com/labs/hpc/How-to-install-CUDA-9-2-on-Ubuntu-18-04-1184/#step-3-install-cuda-dependencies